In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year
from pyspark.sql import functions as F

import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
spark = SparkSession.builder \
    .master("yarn") \
    .config("spark.executor.instances", "4") \
    .config("spark.executor.cores", "4") \
    .config("spark.executor.memory", "16g") \
    .config("spark.driver.memory", "8g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .appName("Graph2_ICUCR") \
    .getOrCreate()

spark

In [ ]:
def cargar_datos(archivo, columnas):
    try:
        return spark.read.parquet(archivo).select(*columnas)
    except Exception as e:
        print(f"Error al procesar {archivo}: {e}")
        return None

dir_segcovid = {
    "df_2020": "hdfs:///rawdata/segcovid/segcovid_parquet_2020",
    "df_2021": "hdfs:///rawdata/segcovid/segcovid_parquet_2021",
    "df_2022": "hdfs:///rawdata/segcovid/segcovid_parquet_2022",
    "df_2023": "hdfs:///rawdata/segcovid/segcovid_parquet_2023"
}

# Columnas de interés
columnas_segcovid = ["PersonaBasicaID", 
                     "AmbitoAtencion",
                     "FechaIngresoAtencion",
                     "FechaEgresoAtencion",
                     "FechaRegistro",
                     "FechaSeguimiento",
                     "DepartamentoAtencion"]

In [ ]:
# Cargar todos los DataFrames
dfs_segcovid = []
for nombre, archivo in dir_segcovid.items():
    df = cargar_datos(archivo, columnas_segcovid)
    if df is not None:
        dfs_segcovid.append(df)

# Concatenar
if dfs_segcovid:
    df_segcovid = dfs_segcovid[0]
    for df in dfs_segcovid[1:]:
        df_segcovid = df_segcovid.unionByName(df, allowMissingColumns=True)

else:
    df_segcovid = None

# Sólo Bogotá
df_segcovid = df_segcovid.filter(df_segcovid['DepartamentoAtencion'] == '11 - Bogotá D.C.')

In [ ]:
## Cargar base ID+SDN+FDN

archivo = "hdfs:///analyticsdata/ID_SDN_FDN"
df_age_sex = spark.read.parquet(archivo)

In [ ]:
## Cálculo de edad en SEGCOVID

df_seg_edad = df_segcovid.join(
    df_age_sex,
    df_segcovid["PersonaBasicaID"] == df_age_sex["personaid"],
    how="left"
).select(
    df_segcovid["*"],
    df_age_sex["FDN"]
).persist()

In [ ]:
df_seg_edad = df_seg_edad.withColumn("Edad", year(col("FechaRegistro")) - col("FDN"))

In [ ]:
df_seg_edad = df_seg_edad.where(col("Edad").isNotNull())

In [ ]:
# Estos pasos de acá los incorporo el 07-03-2025 (temporales)
# df_seg_edad = df_seg_edad.drop_duplicates(["PersonaBasicaID","FechaIngresoAtencion"])
# df_seg_edad = df_seg_edad[df_seg_edad['FechaIngresoAtencion'] != '1900-01-01']

In [ ]:
df_seg_edad.select(["PersonaBasicaID", "AmbitoAtencion", "FechaIngresoAtencion",
                    "FechaEgresoAtencion", "FechaRegistro", "FechaSeguimiento","Edad"]).show()

In [ ]:
# Convertir 'FechaRegistro' a tipo date (formato 'AAAA-MM-DD')
df_seg_edad = df_seg_edad.withColumn("FechaRegistro", F.to_date("FechaRegistro", "yyyy-MM-dd"))
# df_seg_edad = df_seg_edad.withColumn("FechaIngresoAtencion", F.to_date("FechaIngresoAtencion", "yyyy-MM-dd"))

In [ ]:
df_seg_edad = df_seg_edad.withColumn( 
    "GrupoEdad",
    F.when((F.col("Edad") >= -1) & (F.col("Edad") <= 8), "0-9")
     .when((F.col("Edad") >= 9) & (F.col("Edad") <= 18), "10-19")
     .when((F.col("Edad") >= 19) & (F.col("Edad") <= 28), "20-29")
     .when((F.col("Edad") >= 29) & (F.col("Edad") <= 38), "30-39")
     .when((F.col("Edad") >= 39) & (F.col("Edad") <= 48), "40-49")
     .when((F.col("Edad") >= 49) & (F.col("Edad") <= 58), "50-59")
     .when((F.col("Edad") >= 59) & (F.col("Edad") <= 68), "60-69")
     .when((F.col("Edad") >= 69) & (F.col("Edad") <= 78), "70-79")
     .otherwise("80+")
)

In [ ]:
waves = { 
    'Wave 1': ('2020-02-26', '2020-09-25'),
    'Wave 2': ('2020-11-01', '2021-03-01'),
    'Wave 3': ('2021-03-01', '2021-09-14'),
    'Wave 4': ('2021-11-20', '2022-03-24')
}

In [ ]:
from datetime import datetime

# Grafica 
plt.figure(figsize=(5, 5))

labels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80+']
colors = ['#6b6ca3', '#87bcbd', '#6f9954', '#b1615c'] 
i = 0

for wave, (inicio, fin) in waves.items():
    # Convertir fechas a objetos date para usar en los filtros
    inicio_date = datetime.strptime(inicio, '%Y-%m-%d').date()
    fin_date = datetime.strptime(fin, '%Y-%m-%d').date()
    
    # -----------------------------------------------
    # Denominador: total de registros en df_seg_edad por grupo de edad
    df_wave_total = df_seg_edad.filter(
        (F.col("FechaRegistro") >= F.lit(inicio_date)) & (F.col("FechaRegistro") <= F.lit(fin_date))
    )
    
    total_registros_df = df_wave_total.groupBy("GrupoEdad").count().sort("GrupoEdad")
    print(total_registros_df.show())
    
    # Conversión a Pandas para graficar
    grupos_total = total_registros_df.select("GrupoEdad").rdd.flatMap(lambda x: x).collect()
    print(grupos_total)
    dict_total = {'GrupoEdad': grupos_total}
    for columna in total_registros_df.columns:
        valores = total_registros_df.select(columna).rdd.flatMap(lambda x: x).collect()
        print(valores)  
        dict_total[columna] = valores
    
    total_registros_pdf = pd.DataFrame(dict_total)
    total_registros_pdf = total_registros_pdf.set_index("GrupoEdad")["count"].reindex(labels, fill_value=0)
    
    # -----------------------------------------------
    # Numerador: hospitalizados en df_seg_edad por grupo de edad
    df_wave_icu = df_seg_edad.filter(
             (F.col("FechaRegistro") >= F.lit(inicio_date)) &
             (F.col("FechaRegistro") <= F.lit(fin_date)) &
             (F.col("AmbitoAtencion") == "Cuidado Intensivo")
     )

    df_icu_grouped = df_wave_icu.groupBy("GrupoEdad").count().sort("GrupoEdad")
    print(df_icu_grouped.show())
    
    grupos_hosp = df_icu_grouped.select("GrupoEdad").rdd.flatMap(lambda x: x).collect()
    print(grupos_hosp)
    dict_hosp = {'GrupoEdad': grupos_hosp}
    for columna in df_icu_grouped.columns:
        valores = df_icu_grouped.select(columna).rdd.flatMap(lambda x: x).collect()
        print(valores)
        dict_hosp[columna] = valores
    icu_pdf = pd.DataFrame(dict_hosp)
    icu_pdf = icu_pdf.set_index("GrupoEdad")["count"].reindex(labels, fill_value=0)
    

    # Calcular HCR: (hospitalizados / total registros en df_seg_edad)
    ICUCR = icu_pdf / total_registros_pdf
    
    print(icu_pdf)
    print(total_registros_pdf)    
    
    plt.plot(labels, ICUCR, marker='o', linestyle='-', label=wave, color=colors[i])
    i += 1

plt.xlabel('Age Group')
plt.xticks(rotation=90)
plt.ylabel('ICU Case Ratio')
plt.title('Bogotá: ICU CR by Age group Grupo and COVID-19 Waves')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

## Colombia

In [ ]:
# Cargar todos los DataFrames
dfs_segcovid = []
for nombre, archivo in dir_segcovid.items():
    df = cargar_datos(archivo, columnas_segcovid)
    if df is not None:
        dfs_segcovid.append(df)

# Concatenar
if dfs_segcovid:
    df_segcovid = dfs_segcovid[0]
    for df in dfs_segcovid[1:]:
        df_segcovid = df_segcovid.unionByName(df, allowMissingColumns=True)

else:
    df_segcovid = None

# Sólo Colombia
df_col = df_segcovid

In [ ]:
## Cargar base ID+SDN+FDN

archivo = "hdfs:///analyticsdata/ID_SDN_FDN"
df_age_sex = spark.read.parquet(archivo)

In [ ]:
## Cálculo de edad en SEGCOVID

df_seg_edad = df_col.join(
    df_age_sex,
    df_col["PersonaBasicaID"] == df_age_sex["personaid"],
    how="left"
).select(
    df_col["*"],
    df_age_sex["FDN"]
).persist()

In [ ]:
df_seg_edad = df_seg_edad.withColumn("Edad", year(col("FechaRegistro")) - col("FDN"))

In [ ]:
df_seg_edad = df_seg_edad.where(col("Edad").isNotNull())

In [ ]:
# Convertir 'FechaRegistro' a tipo date (formato 'AAAA-MM-DD')
df_seg_edad = df_seg_edad.withColumn("FechaRegistro", F.to_date("FechaRegistro", "yyyy-MM-dd"))

In [ ]:
df_seg_edad = df_seg_edad.withColumn( 
    "GrupoEdad",
    F.when((F.col("Edad") >= -1) & (F.col("Edad") <= 8), "0-9")
     .when((F.col("Edad") >= 9) & (F.col("Edad") <= 18), "10-19")
     .when((F.col("Edad") >= 19) & (F.col("Edad") <= 28), "20-29")
     .when((F.col("Edad") >= 29) & (F.col("Edad") <= 38), "30-39")
     .when((F.col("Edad") >= 39) & (F.col("Edad") <= 48), "40-49")
     .when((F.col("Edad") >= 49) & (F.col("Edad") <= 58), "50-59")
     .when((F.col("Edad") >= 59) & (F.col("Edad") <= 68), "60-69")
     .when((F.col("Edad") >= 69) & (F.col("Edad") <= 78), "70-79")
     .otherwise("80+")
)

In [ ]:
waves = { 
    'Ola 1': ('2020-02-26', '2020-09-25'),
    'Ola 2': ('2020-11-01', '2021-03-01'),
    'Ola 3': ('2021-03-01', '2021-09-14'),
    'Ola 4': ('2021-11-20', '2022-03-24')
}

In [ ]:
from datetime import datetime
import math

def wilson_interval(num, tot, z=1.96):
    if tot == 0:
        return 0, 0
    p = num / tot
    denominator = 1 + (z**2) / tot
    center = p + (z**2) / (2*tot)
    margin = z * math.sqrt((p*(1-p)/tot) + (z**2)/(4*(tot**2)))
    lower = (center - margin) / denominator
    upper = (center + margin) / denominator
    return lower, upper

# Grafica 
plt.figure(figsize=(5, 5))

labels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80+']
colors = ['#6a5acd', '#66c2a5', '#4daf4a', '#d95f02']
i = 0

for wave, (inicio, fin) in waves.items():
    # Convertir fechas a objetos date para usar en los filtros
    inicio_date = datetime.strptime(inicio, '%Y-%m-%d').date()
    fin_date = datetime.strptime(fin, '%Y-%m-%d').date()
    
    # -----------------------------------------------
    # Denominador: total de registros en df_seg_edad por grupo de edad
    df_wave_total = df_seg_edad.filter(
        (F.col("FechaRegistro") >= F.lit(inicio_date)) & (F.col("FechaRegistro") <= F.lit(fin_date))
    )
    
    total_registros_df = df_wave_total.groupBy("GrupoEdad").count().sort("GrupoEdad")
    
    # Conversión a Pandas para graficar
    grupos_total = total_registros_df.select("GrupoEdad").rdd.flatMap(lambda x: x).collect()
    dict_total = {'GrupoEdad': grupos_total}
    for columna in total_registros_df.columns:
        valores = total_registros_df.select(columna).rdd.flatMap(lambda x: x).collect()
        dict_total[columna] = valores
        
    total_registros_pdf = pd.DataFrame(dict_total)
    total_registros_pdf = total_registros_pdf.set_index("GrupoEdad")["count"].reindex(labels, fill_value=0)
    
    
    df_wave_icu = df_seg_edad.filter(
             (F.col("FechaRegistro") >= F.lit(inicio_date)) &
             (F.col("FechaRegistro") <= F.lit(fin_date)) &
             (F.col("AmbitoAtencion") == "Cuidado Intensivo")
     )

    df_icu_grouped = df_wave_icu.groupBy("GrupoEdad").count().sort("GrupoEdad")
    
    grupos_hosp = df_icu_grouped.select("GrupoEdad").rdd.flatMap(lambda x: x).collect()

    dict_hosp = {'GrupoEdad': grupos_hosp}
    for columna in df_icu_grouped.columns:
        valores = df_icu_grouped.select(columna).rdd.flatMap(lambda x: x).collect()
        dict_hosp[columna] = valores
    icu_pdf = pd.DataFrame(dict_hosp)
    icu_pdf = icu_pdf.set_index("GrupoEdad")["count"].reindex(labels, fill_value=0)

    # Calcular ICUCR: (icu_pdf / total registros en df_seg_edad)
    ICUCR = 100 * (icu_pdf / total_registros_pdf)
    
    # Calcular los errores de cada barra usando el intervalo de Wilson
    lower_errors = []
    upper_errors = []
    for grupo in labels:
        num = icu_pdf.get(grupo, 0)
        tot = total_registros_pdf.get(grupo, 0)
        if tot > 0:
            lower, upper = wilson_interval(num, tot)
            p = num / tot
            # Convertir a porcentaje y calcular el error inferior y superior
            lower_errors.append(100 * (p - lower))
            upper_errors.append(100 * (upper - p))
        else:
            lower_errors.append(0)
            upper_errors.append(0)
    
    print(icu_pdf)
    print(total_registros_pdf) 
    
    # Graficar la línea con sus barras de error
    plt.errorbar(labels, ICUCR, yerr=[lower_errors, upper_errors],
                 fmt='o-', color=colors[i], ecolor='black', capsize=5,
                 label=wave)
    i += 1
    
#     plt.plot(labels, ICUCR, marker='o', linestyle='-', label=wave, color=colors[i])

plt.xlabel('Grupo de edad', fontsize=12)
plt.xticks(rotation=45)
plt.ylabel('Porcentaje', fontsize=12)
plt.title('Riesgo de ingreso a UCI', fontsize=14)
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)

# Formatear el eje y para mostrar porcentajes
from matplotlib.ticker import PercentFormatter
plt.gca().yaxis.set_major_formatter(PercentFormatter(decimals=0))

plt.tight_layout()
plt.show()

---
---
---
## Análisis para todos los departamentos

In [ ]:
from pyspark.sql import Window
from pyspark.sql.functions import col, to_date, datediff, when, row_number, avg
import pyspark.sql.functions as F
import pandas as pd

# -------------------------------
# 1. Cargar y unir los DataFrames de SEGCOVID
dfs_segcovid = []
for nombre, archivo in dir_segcovid.items():
    df = cargar_datos(archivo, columnas_segcovid)
    if df is not None:
        dfs_segcovid.append(df)

if dfs_segcovid:
    df_segcovid = dfs_segcovid[0]
    for df in dfs_segcovid[1:]:
        df_segcovid = df_segcovid.unionByName(df, allowMissingColumns=True)
else:
    df_segcovid = None

df_segcovid = df_segcovid.filter(col("DepartamentoAtencion") != "-1 - NO DEFINIDO")

df_segcovid_v2 = df_segcovid.withColumn(
    'DepartamentoAtencion',
     when(df_segcovid['DepartamentoAtencion'] == '11 - Bogotá D.C.', 'DC')
    .when(df_segcovid['DepartamentoAtencion'] == '05 - Antioquia', 'ANT')
    .when(df_segcovid['DepartamentoAtencion'] == '76 - Valle del Cauca', 'VAC')
    .when(df_segcovid['DepartamentoAtencion'] == '08 - Atlántico', 'ATL')
    .when(df_segcovid['DepartamentoAtencion'] == '25 - Cundinamarca', 'CUN')
    .when(df_segcovid['DepartamentoAtencion'] == '68 - Santander', 'SAN')
    .when(df_segcovid['DepartamentoAtencion'] == '13 - Bolívar', 'BOL')
    .when(df_segcovid['DepartamentoAtencion'] == '47 - Magdalena', 'MAG')
    .when(df_segcovid['DepartamentoAtencion'] == '15 - Boyacá', 'BOY')
    .when(df_segcovid['DepartamentoAtencion'] == '17 - Caldas', 'CAL')
    .when(df_segcovid['DepartamentoAtencion'] == '52 - Nariño', 'NAR')
    .when(df_segcovid['DepartamentoAtencion'] == '20 - Cesar', 'CES')
    .when(df_segcovid['DepartamentoAtencion'] == '23 - Córdoba', 'COR')
    .when(df_segcovid['DepartamentoAtencion'] == '73 - Tolima', 'TOL')
    .when(df_segcovid['DepartamentoAtencion'] == '50 - Meta', 'MET')
    .when(df_segcovid['DepartamentoAtencion'] == '41 - Huila', 'HUI')
    .when(df_segcovid['DepartamentoAtencion'] == '54 - Norte de Santander', 'NSA')
    .when(df_segcovid['DepartamentoAtencion'] == '66 - Risaralda', 'RIS')
    .when(df_segcovid['DepartamentoAtencion'] == '19 - Cauca', 'CAU')
    .when(df_segcovid['DepartamentoAtencion'] == '44 - La Guajira', 'LAG')
    .when(df_segcovid['DepartamentoAtencion'] == '70 - Sucre', 'SUC')
    .when(df_segcovid['DepartamentoAtencion'] == '63 - Quindio', 'QUI')
    .when(df_segcovid['DepartamentoAtencion'] == '85 - Casanare', 'CAS')
    .when(df_segcovid['DepartamentoAtencion'] == '18 - Caquetá', 'CAQ')
    .when(df_segcovid['DepartamentoAtencion'] == '86 - Putumayo', 'PUT')
    .when(df_segcovid['DepartamentoAtencion'] == '27 - Chocó', 'CHO')
    .when(df_segcovid['DepartamentoAtencion'] == '81 - Arauca', 'ARA')
    .when(df_segcovid['DepartamentoAtencion'] == '91 - Amazonas', 'AMA')
    .when(df_segcovid['DepartamentoAtencion'] == '88 - Archipiélago de San Andrés Providencia y Santa Catalina', 'SAP')
    .when(df_segcovid['DepartamentoAtencion'] == '95 - Guaviare', 'GUV')
    .when(df_segcovid['DepartamentoAtencion'] == '99 - Vichada', 'VIC')
    .when(df_segcovid['DepartamentoAtencion'] == '94 - Guainía', 'GUA')
    .when(df_segcovid['DepartamentoAtencion'] == '97 - Vaupés', 'VAU')
    .otherwise(df_segcovid['DepartamentoAtencion'])
)

df_deptos = df_segcovid_v2

In [ ]:
## Cargar base ID+SDN+FDN

archivo = "hdfs:///analyticsdata/ID_SDN_FDN"
df_age_sex = spark.read.parquet(archivo)

## Cálculo de edad en SEGCOVID

df_seg_edad = df_deptos.join(
    df_age_sex,
    df_deptos["PersonaBasicaID"] == df_age_sex["personaid"],
    how="left"
).select(
    df_deptos["*"],
    df_age_sex["FDN"]
).persist()

df_seg_edad = df_seg_edad.withColumn("Edad", year(col("FechaRegistro")) - col("FDN"))

df_seg_edad = df_seg_edad.where(col("Edad").isNotNull())

# Convertir 'FechaRegistro' a tipo date (formato 'AAAA-MM-DD')
df_seg_edad = df_seg_edad.withColumn("FechaRegistro", F.to_date("FechaRegistro", "yyyy-MM-dd"))

df_seg_edad = df_seg_edad.withColumn( 
    "GrupoEdad",
    F.when((F.col("Edad") >= -1) & (F.col("Edad") <= 8), "0-9")
     .when((F.col("Edad") >= 9) & (F.col("Edad") <= 18), "10-19")
     .when((F.col("Edad") >= 19) & (F.col("Edad") <= 28), "20-29")
     .when((F.col("Edad") >= 29) & (F.col("Edad") <= 38), "30-39")
     .when((F.col("Edad") >= 39) & (F.col("Edad") <= 48), "40-49")
     .when((F.col("Edad") >= 49) & (F.col("Edad") <= 58), "50-59")
     .when((F.col("Edad") >= 59) & (F.col("Edad") <= 68), "60-69")
     .when((F.col("Edad") >= 69) & (F.col("Edad") <= 78), "70-79")
     .otherwise("80+")
)

waves = { 
    'Wave 1': ('2020-02-26', '2020-09-25'),
    'Wave 2': ('2020-11-01', '2021-03-01'),
    'Wave 3': ('2021-03-01', '2021-09-14'),
    'Wave 4': ('2021-11-20', '2022-03-24')
}

In [ ]:
labels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80+']
colors = ['#6b6ca3', '#87bcbd', '#6f9954', '#b1615c'] 

In [ ]:
from datetime import datetime
import pandas as pd
from pyspark.sql import functions as F

# Obtener lista de departamentos únicos
departamentos = [row["DepartamentoAtencion"] for row in df_seg_edad.select("DepartamentoAtencion").distinct().collect()]

resultados = []  # Lista para almacenar los resultados finales

# Iterar sobre cada departamento
for dept in departamentos:
    # Filtrar los registros para el departamento actual
    df_dept = df_seg_edad.filter(F.col("DepartamentoAtencion") == dept)
    
    # Iterar sobre cada wave
    for wave, (inicio, fin) in waves.items():
        # Convertir fechas a objetos date
        inicio_date = datetime.strptime(inicio, '%Y-%m-%d').date()
        fin_date = datetime.strptime(fin, '%Y-%m-%d').date()
        
        # ----------------------------
        # Cálculo del denominador: Total de registros por grupo de edad para el departamento y wave actual
        df_wave_total = df_dept.filter(
            (F.col("FechaRegistro") >= F.lit(inicio_date)) &
            (F.col("FechaRegistro") <= F.lit(fin_date))
        )
        total_registros_df = df_wave_total.groupBy("GrupoEdad").count().sort("GrupoEdad")
        
        grupos_total = total_registros_df.select("GrupoEdad").rdd.flatMap(lambda x: x).collect()
        dict_total = {'GrupoEdad': grupos_total}
        for columna in total_registros_df.columns:
            valores = total_registros_df.select(columna).rdd.flatMap(lambda x: x).collect()
            dict_total[columna] = valores
        total_registros_pdf = pd.DataFrame(dict_total)
        total_registros_pdf = total_registros_pdf.set_index("GrupoEdad")["count"].reindex(labels, fill_value=0)
        
        # ----------------------------
        # Cálculo del numerador: Casos en ICU para el departamento y wave actual
        df_wave_icu = df_dept.filter(
            (F.col("FechaRegistro") >= F.lit(inicio_date)) &
            (F.col("FechaRegistro") <= F.lit(fin_date)) &
            (F.col("AmbitoAtencion") == "Cuidado Intensivo")
        )
        df_icu_grouped = df_wave_icu.groupBy("GrupoEdad").count().sort("GrupoEdad")
        
        grupos_icu = df_icu_grouped.select("GrupoEdad").rdd.flatMap(lambda x: x).collect()
        dict_icu = {'GrupoEdad': grupos_icu}
        for columna in df_icu_grouped.columns:
            valores = df_icu_grouped.select(columna).rdd.flatMap(lambda x: x).collect()
            dict_icu[columna] = valores
        icu_pdf = pd.DataFrame(dict_icu)
        icu_pdf = icu_pdf.set_index("GrupoEdad")["count"].reindex(labels, fill_value=0)
        
        # ----------------------------
        # Calcular la tasa ICUCR: (casos ICU / total registros)
        ICUCR = icu_pdf / total_registros_pdf
        ICUCR = ICUCR.fillna(0)  # Para evitar NaN en caso de división por cero
        
        # Guardar los resultados para cada grupo de edad
        for grupo in labels:
            total_val = int(total_registros_pdf.loc[grupo])
            icu_val = int(icu_pdf.loc[grupo])
            icucr_val = float(ICUCR.loc[grupo])
            resultados.append((dept, grupo, total_val, icu_val, icucr_val, wave))

# Convertir la lista de resultados en un DataFrame de Pandas
# resultados_pdf = pd.DataFrame(resultados, columns=["Departamento", "GrupoEdad", "Total", "ICU", "ICUCR", "Wave"])

# Mostrar las primeras filas del resultado
# print(resultados_pdf.head())


In [ ]:
type(resultados)
resultados

In [ ]:
df_result= pd.DataFrame(resultados,  columns =["DepartamentoNotificacion", "GrupoEdad", "Total", "ICU", "ICUCR", "Wave"]).sort_values(by=["DepartamentoNotificacion"])

In [ ]:
waves_list = list(waves.keys())

# Grafica
fig, axs = plt.subplots(2, 2, figsize=(14, 16))
axs = axs.flatten()

# Factor de escala burbujas
scale_factor = 1200

for i, wave in enumerate(waves_list):
    df_wave = df_result[df_result['Wave'] == wave]
    scatter = axs[i].scatter(
        x=df_wave['GrupoEdad'].astype(str),
        y=df_wave['DepartamentoNotificacion'],
        s=df_wave['ICUCR'] * scale_factor,
        alpha=0.6,
        c=df_wave['ICUCR'],
        cmap='viridis'
    )
    axs[i].set_title(f'ICUCR - {wave}')
    axs[i].set_xlabel('Age Group')
    axs[i].set_ylabel('Department')
    
    axs[i].tick_params(axis='x', labelrotation=45)
    
    axs[i].invert_yaxis()
    
    cbar = plt.colorbar(scatter, ax=axs[i])
#     cbar.set_label('HCR', rotation=0, y=1.0)

plt.tight_layout()
plt.show()